# Summary

Thanks for Dave to share the great PMC vectorization file: http://evexdb.org/pmresources/vec-space-models/

The vectorization part is developed from the tutorials by Francois Chollet.

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from gensim.models import KeyedVectors
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.optimizers import Adam



MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 200

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 Vectorization

In [2]:
embeddings_index = KeyedVectors.load_word2vec_format('PubMed-and-PMC-w2v.bin', binary=True)

In [3]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [4]:
p_file = 'RCT_labels'

with open(p_file, 'rb') as fin:
    labels = pickle.load(fin)

In [5]:
# vectorize the text into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(data_X)
sequences = tokenizer.texts_to_sequences(data_X)

In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 1805160 unique tokens.
Shape of data tensor: (19111, 1000)
Shape of label tensor: (19111, 2)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)

In [8]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    try:
        embedding_vector = embeddings_index[word]
    except KeyError:
        continue
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


Preparing embedding matrix.


# 2 LSTM

In [9]:
def model():
        
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(embedded_sequences)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = sequence_input, outputs = X)
        
    return model

In [10]:
model = model()

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 200)         4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 247, 768)          2304768   
_________________________________________________________________
batch_normalization_1 (Batch (None, 247, 768)          3072      
_________________________________________________________________
activation_1 (Activation)    (None, 247, 768)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 247, 768)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 247, 512)          2623488   
__________

In [12]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [13]:
model.fit(X_train, Y_train, batch_size = 64, epochs=50)

Epoch 1/50
14333/14333 [==============================] - 290s 20ms/step - loss: 0.4215 - acc: 0.8171
Epoch 2/50
14333/14333 [==============================] - 286s 20ms/step - loss: 0.3669 - acc: 0.8510
Epoch 3/50
14333/14333 [==============================] - 285s 20ms/step - loss: 0.3493 - acc: 0.8589
Epoch 4/50
14333/14333 [==============================] - 284s 20ms/step - loss: 0.3331 - acc: 0.8635
Epoch 5/50
14333/14333 [==============================] - 285s 20ms/step - loss: 0.3162 - acc: 0.8693
Epoch 6/50
14333/14333 [==============================] - 284s 20ms/step - loss: 0.3027 - acc: 0.8762
Epoch 7/50
14333/14333 [==============================] - 285s 20ms/step - loss: 0.2834 - acc: 0.8822
Epoch 8/50
14333/14333 [==============================] - 285s 20ms/step - loss: 0.2719 - acc: 0.8892
Epoch 9/50
14333/14333 [==============================] - 285s 20ms/step - loss: 0.2497 - acc: 0.8986
Epoch 10/50
14333/14333 [==============================] - 284s 20ms/step - loss: 

In [14]:
model_json = model.to_json()
with open ("LSTMModel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModel.h5")

In [15]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 39s 8ms/step
Validation set accuracy =  0.8302637086397636
